In this notebook I am going to reproduce the experiment but using an approximation to the identity instead of a smoothing kernel.

In [1]:
import numpy as np
from numpy import zeros, eye, array, r_, diag, ones, log, zeros_like, pi, exp, log10, sort, quantile, linspace, vstack
from numpy import save, nanmean
from numpy.random import randn, rand, randint, default_rng
from numpy.random import normal
from numpy import isfinite
from numpy.linalg import solve, det, inv, norm
from numpy import logspace

import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns

from collections import OrderedDict
import copy

from warnings import catch_warnings, filterwarnings

from autograd import jacobian, grad
import autograd.numpy as anp
from autograd.numpy import ones as aones
from autograd.numpy import zeros as azeros
from autograd.numpy import exp as aexp
from autograd.numpy.linalg import det as adet
from autograd.numpy import log as alog
from autograd.numpy import diag as adiag
from autograd.numpy.linalg import norm as anorm
import autograd.scipy as asp
from autograd.scipy.stats import norm as andist
import autograd as ag
from autograd.scipy.special import gamma as agamma

from scipy.stats import multivariate_normal as MVN
from scipy.optimize import fsolve, bisect
from scipy.stats import norm as ndist
from scipy.stats import uniform as udist
from scipy.stats import beta as abetadist
from scipy.linalg import qr, lstsq
from scipy.special import gamma

from g_and_k_functions import GandK
from tangential_hug_functions import HugTangentialMultivariate
from RWM import RWM
from HMC.static_hmc import HMC
from utils import ESS_univariate, ESS
import time

from Manifolds.Manifold import Manifold
# from Zappa.zappa import zappa_sampling_storecomps

from itertools import product
import scipy.linalg as la

import math
from copy import deepcopy
from arviz import convert_to_dataset
from arviz import ess as ess_arviz
from utils import ESS_univariate, compute_arviz_miness_runtime
from Zappa.zappa import zappa_sampling_storecomps_rattle_manifold, project_zappa_manifold
from utils import generate_powers_of_ten

In [2]:
# Parameters
θ0        = array([3.0, 1.0, 2.0, 0.5]) # True parameter value
m         = 50                          # Number of latent variables
d         = 4 + m                       # Dimensionality of ξ=(θ, z)
# seed=1234

In [3]:
def f(ξ):
    """Deterministic simulator f: ξ -> y."""
    return ξ[0] + ξ[1]*(1 + 0.8*(1 - aexp(-ξ[2]*ξ[4:]))/(1 + aexp(-ξ[2]*ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3])*ξ[4:]

def data_generator(θ0, m, seed):
    """Stochastic Simulator. Generates y given θ."""
    rng = default_rng(seed)
    z = rng.normal(size=m)
    return f(r_[θ0, z])

def Jf_transpose(ξ):
    """Transpose of the Jacobian of f."""
    return vstack((
        aones(len(ξ[4:])),
        (1 + 0.8 * (1 - aexp(-ξ[2] * ξ[4:])) / (1 + aexp(-ξ[2] * ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3]) * ξ[4:],
        8 * ξ[1] * (ξ[4:]**2) * ((1 + ξ[4:]**2)**ξ[3]) * aexp(ξ[2]*ξ[4:]) / (5 * (1 + aexp(ξ[2]*ξ[4:]))**2),
        ξ[1]*ξ[4:]*((1+ξ[4:]**2)**ξ[3])*(1 + 9*aexp(ξ[2]*ξ[4:]))*alog(1 + ξ[4:]**2) / (5*(1 + aexp(ξ[2]*ξ[4:]))),
        adiag(ξ[1]*((1+ξ[4:]**2)**(ξ[3]-1))*(((18*ξ[3] + 9)*(ξ[4:]**2) + 9)*aexp(2*ξ[2]*ξ[4:]) + (8*ξ[2]*ξ[4:]**3 + (20*ξ[3] + 10)*ξ[4:]**2 + 8*ξ[2]*ξ[4:] + 10)*aexp(ξ[2]*ξ[4:]) + (2*ξ[3] + 1)*ξ[4:]**2 + 1) / (5*(1 + aexp(ξ[2]*ξ[4:]))**2))
    ))

def Jf(ξ):
    """Jacobian of f."""
    return Jf_transpose(ξ).T

def check_jacobian(tol=1e-12):
    """Checks if Jacobian function is correct."""
    ξ = r_[θ0, randn(m)]
    return np.max(abs(Jf(ξ) - jacobian(f)(ξ))) < tol

Jf_autograd = jacobian(f)

def log_pθ_beta(θ):
    """Beta logprior for θ."""
    assert len(θ) == 4, "θ must be 4-dimensional but has length {}".format(len(θ))
    #return abetadist.logpdf(θ, a=2, b=2, scale=10).sum()
    # α=2.0 and β=2.0
    return (alog(θ/10) + alog(1-θ/10) -alog(2) -alog(2) + alog(agamma(4))- alog(10)).sum()
    
    
def log_pz(z):
    """Normal logprior for z."""
    return andist.logpdf(z).sum()

def logprior_beta(ξ):
    """Log prior on ξ using Beta(2,2) for θ."""
    return log_pθ_beta(ξ[:4]) + log_pz(ξ[4:])

def neg_logprior_beta(ξ):
    return - logprior_beta(ξ)

def sample_θ_beta():
    """Samples θ from Beta prior. """
    return abetadist.rvs(a=2, b=2, scale=10, size=4) #10*abetadist.rvs(a=2, b=2, scale=1, size=4)

def sample_prior_beta():
    """Samples from prior for ξ where p(θ) is a Beta(2,2)."""
    return anp.r_[sample_θ_beta(), randn(m)]

def log_normal_kernel(ξ, ystar, ϵ):
    """Log normal kernel density."""
    u = anorm(f(ξ) - ystar)
    return -u**2/(2*(ϵ**2)) -0.5*alog(2*pi*(ϵ**2))

def log_abc_posterior_beta(ξ, ystar, ϵ):
    """Log posterior density. """
    return logprior_beta(ξ) + log_normal_kernel(ξ, ystar, ϵ)

# def find_point_on_manifold(ystar, ϵ, max_iter=1000):
#     """Find a point on the data manifold."""
#     i = 0
#     with catch_warnings():
#         filterwarnings('error')
#         while i <= max_iter:
#             i += 1
#             try: 
#                 ξ_guess = sample_prior_beta()
#                 ξ_found = fsolve(lambda ξ: r_[f(ξ) - ystar, azeros(4)], ξ_guess)
#                 if not isfinite([log_abc_posterior_beta(ξ_found, ystar, ϵ)]):
#                     pass
#                 else:
#                     return ξ_found

#             except RuntimeWarning:
#                 continue
#         raise ValueError("Couldn't find a point, try again.")

def find_point_on_manifold(ystar, ϵ, max_iter=1000, tol=1.49012e-08, m=50):
    """Find a point on the data manifold."""
    i = 0
    with catch_warnings():
        filterwarnings('error')
        while i <= max_iter:
            i += 1
            try: 
                # Sample θ from the prior
                θfixed = sample_θ_beta()
                function = lambda z: f(r_[θfixed, z]) - ystar
                z_guess  = randn(m)
                z_found  = fsolve(function, z_guess, xtol=tol)
                ξ_found  = r_[θfixed, z_found]
                if not isfinite([log_abc_posterior_beta(ξ_found, ystar, ϵ)]):
                    pass
                else:
                    return ξ_found

            except RuntimeWarning:
                continue
        raise ValueError("Couldn't find a point, try again.")    
        
def find_point_on_manifold_from_θ(ystar, θfixed, ϵ, max_iter=2000, tol=1.49012e-08):
    """Same as the above but we provide the θfixed. Can be used to find a point where
    the theta is already θ0."""
    function = lambda z: f(r_[θfixed, z]) - ystar
    z_guess  = randn(m)
    z_found  = fsolve(function, z_guess, xtol=tol)
    ξ_found  = r_[θfixed, z_found]
    if not isfinite([log_abc_posterior_beta(ξ_found, ystar, ϵ)]):
        raise ValueError("Couldn't find a point.")
    else:
        return ξ_found

def is_on_manifold(ξ, ystar, tol=1e-8):
    """Checks if ξ is on the ystar manifold."""
    return np.max(abs(f(ξ) - ystar)) < tol
        
q = MVN(zeros(d), eye(d))

In [4]:
ystar = data_generator(θ0, m, seed=1234)

In [5]:
def generate_powers_of_ten(max_exponent, min_exponent):
    """E.g. generate_powers_of_ten(2, -1) will return 100, 10, 0, 0.1."""
    number_of_powers = max_exponent + abs(min_exponent) + 1
    return logspace(start=max_exponent, stop=min_exponent, num=number_of_powers, endpoint=True)

In [6]:
ξ0_found = find_point_on_manifold(ystar, ϵ=1e-5, max_iter=5000, tol=1e-16)

In [7]:
def run_thug(ξ0, T, B, N_samples, α, ystar, ϵ, method='qr'):
    """Runs THUG with various settings."""
    start_time = time.time()
    thug_samples, thug_acc = HugTangentialMultivariate(
        x0=ξ0,
        T=T,
        B=B,
        N=N_samples,
        α=α,
        q=q,
        logpi=lambda ξ: log_abc_posterior_beta(ξ, ystar, ϵ=ϵ),
        jac=Jf,
        method=method
    )
    runtimeTHUG = time.time() - start_time
    return min(ESS_univariate(thug_samples)) / runtimeTHUG, thug_acc.mean()

def run_crwm(ξ0, T, B, N_samples, ystar, tol=1e-10, rev_tol=1e-10, norm_ord=2):
    """Runs C-RWM with various settings."""
    manifold = GKManifold(ystar=ystar)
    # Run
    start_time = time.time()
    crwm_samples, crwm_evals, crwm_acc  = zappa_sampling_storecomps_rattle(ξ0, manifold, N_samples, T, B, tol=tol, rev_tol=rev_tol, norm_ord=norm_ord)
    runtimeCRWM = time.time() - start_time
    return min(ESS_univariate(crwm_samples)) / runtimeCRWM, crwm_acc.mean()

In [8]:
def run_thug_for_various_ϵ_and_B(ϵs, Bs, δ, N_samples, α=0.0, method='qr'):
    """For each ϵ and for each B it runs THUG and computes its minESS/runtime."""
    THUG_CC = zeros((len(ϵs), len(Bs)))
    THUG_AP = zeros((len(ϵs), len(Bs)))
    for ϵ_ix, ϵ in enumerate(ϵs):
        for B_ix, B in enumerate(Bs):
            print("ϵ = ", ϵ, " B = ", B)
            thug_cc, thug_ap = run_thug(ξ0_found, B*δ, B, N_samples, α, ystar, ϵ, method=method)
            THUG_CC[ϵ_ix, B_ix] = thug_cc
            THUG_AP[ϵ_ix, B_ix] = thug_ap
    return THUG_CC, THUG_AP

def avg_thug_for_various_ϵ_and_B(ϵs, Bs, δ, N_samples, α, method, n_runs):
    """Basically averages results of `run_thug_for_various_ϵ_and_B` for several runs."""
    THUG_CC = zeros((len(ϵs), len(Bs)))
    THUG_AP = zeros((len(ϵs), len(Bs)))
    for i in range(n_runs):
        thug_cc, thug_ap = run_thug_for_various_ϵ_and_B(ϵs, Bs, δ, N_samples, α=α, method=method)
        THUG_CC += (thug_cc / n_runs)
        THUG_AP += (thug_ap / n_runs)
    return THUG_CC, THUG_AP

def run_crwm_for_various_B(Bs, δ, N_samples, tol=1e-16, rev_tol=1e-16, norm_ord=2):
    """Basically same as above but since C-RWM is on the manifold we don't need to run it for different ϵs."""
    CRWM_CC = zeros(len(Bs))
    CRWM_AP = zeros(len(Bs))
    for B_ix, B in enumerate(Bs):
        print("B = ", B)
        crwm_cc, crwm_ap = run_crwm(ξ0_found, B*δ, B, N_samples, ystar, tol=tol, rev_tol=rev_tol, norm_ord=norm_ord)
        CRWM_CC[B_ix] = crwm_cc
        CRWM_AP[B_ix] = crwm_ap
    return CRWM_CC, CRWM_AP

def avg_crwm_for_various_B(Bs, δ, N_samples, n_runs, tol=1e-16, rev_tol=1e-16):
    """Averages `run_crwm_for_various_B` for a number of runs."""
    CRWM_CC = zeros(len(Bs))
    CRWM_AP = zeros(len(Bs))
    for i in range(n_runs):
        crwm_cc, crwm_ap = run_crwm_for_various_B(Bs, δ, N_samples, tol=tol, rev_tol=rev_tol)
        CRWM_CC += (crwm_cc / n_runs)
        CRWM_AP += (crwm_ap / n_runs)
    return CRWM_CC, CRWM_AP

In [9]:
# ϵs        = generate_powers_of_ten(0, -4) #generate_powers_of_ten(2, -8)[::2]
# δ         = 0.05 #0.01
# N_samples = 10000
# Bs        = [5, 10, 20]

# New

In [ ]:
class GKManifold(Manifold):
    def __init__(self, ystar):
        self.m = len(ystar)            # Number constraints = dimensionality of the data
        self.d = 4                     # Manifold has dimension 4 (like the parameter θ)
        self.n = self.d + self.m       # Dimension of ambient space is m + 4
        self.ystar = ystar

    def q(self, ξ):
        """Constraint for toy BIP."""
        with catch_warnings():
            filterwarnings('error')
            try:
                return (ξ[0] + ξ[1]*(1 + 0.8*(1 - exp(-ξ[2]*ξ[4:]))/(1 + exp(-ξ[2]*ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3])*ξ[4:]) - self.ystar
            except RuntimeWarning:
                raise ValueError("Constraint found Overflow warning.")

    def Q(self, ξ):
        """Transpose of Jacobian for toy BIP. """
        return Jf_transpose(ξ)
    
    def compute_J(self, ξ):
        """Safely computes Jacobian."""
        with catch_warnings():
            filterwarnings('error')
            try:
                return Jf(ξ)
            except RuntimeWarning:
                raise ValueError("J computation found Runtime warning.")
                
    def logprior(self, ξ):
        """Log prior distribution."""
        θ, z = ξ[:4], ξ[4:]
        with catch_warnings():
            filterwarnings('error')
            try:
                theta_prior_term = (np.log(θ/10) + np.log(1-θ/10)).sum()
                z_prior_term = -z@z/2
                return theta_prior_term + z_prior_term
            except RuntimeWarning:
                return -np.inf

    def logpost(self, ξ):
        """log posterior for c-rwm"""
        θ, z = ξ[:4], ξ[4:]
        try:
            J = self.compute_J(ξ)
            logprior = self.logprior(ξ)
            correction_term  = - math.prod(np.linalg.slogdet(J@J.T))/2 
            return  logprior + correction_term
        except ValueError as e:
            return -np.inf
            
#         if Jsuccessful:
#             theta_prior_term = (np.log(θ/10) + np.log(1-θ/10)).sum()
#             z_prior_term     = - z@z/2
#             correction_term  = - math.prod(np.linalg.slogdet(J@J.T))/2 
#             return  theta_prior_term + z_prior_term + correction_term
#         else:
#             return -np.inf
        #return (np.log(θ/10) + np.log(1-θ/10)).sum() - np.log(det(J@J.T))/2 -z@z/2
    
    def logp(self, v):
        """Log density for normal on the tangent space."""
        return MVN(mean=zeros(self.d), cov=eye(self.d)).logpdf(v)

#### Common Settings

In [10]:
N_CHAINS  = 1
N_SAMPLES = 100
BS        = [1, 10]
αs        = [0.0, 0.9]
ϵs        = generate_powers_of_ten(0, -4)
δ         = 0.01

#### Experimental Settings

In [ ]:
def generate_settings(N, δ, Ns, Bs, ϵs, seeds, n_chains=4, u1_true=True, tol=1e-15, maxiter=5000):
    """Generates variables for the experiment."""
    manifold = LVManifold(Ns=Ns, n_chains=n_chains, seeds=seeds)
    u0s = manifold.find_init_points_for_each_chain(u1_true=u1_true, tol=tol, maxiter=maxiter)
    return {
        'N': N,
        'δ': δ,
        'Ns': Ns,
        'Bs': Bs,
        'ϵs': ϵs,
        'u0s': u0s,
        'manifold': manifold,
        'n_chains': n_chains
    }

In [ ]:
def cc_experiment_thug(settings, α=0.0, verbose=False):
    """Computational Cost of THUG and C-RWM."""
    verboseprint = print if verbose else lambda *a, **k: None
    ϵs, Bs = settings['ϵs'], settings['Bs']
    u0s = settings['u0s']
    δ = settings['δ']
    N = settings['N']
    q = settings['manifold'].q_dist
    J = settings['manifold'].J
    n_chains = settings['n_chains']
    ESS_TABLE = zeros((len(ϵs), len(Bs)))
    AP_TABLE  = zeros((len(ϵs), len(Bs)))
    for ϵ_ix, ϵ in enumerate(ϵs):
        logηϵ = settings['manifold'].generate_logpi(ϵ)
        for B_ix, B in enumerate(Bs):
            chains = []
            times  = []
            avg_ap = 0.0
            for chain_ix in range(n_chains):
                start_time = time.time()
                s, a = HugTangentialMultivariateSafe(u0s[chain_ix, :], B*δ, B, N, α, q, logηϵ, J)
                runtime = time.time() - start_time
                verboseprint("B={} time={} a={}".format(B, runtime, a.mean()))
                chains.append(s)
                times.append(runtime)
                avg_ap += (a.mean() / n_chains)
            ESS_TABLE[ϵ_ix, B_ix] = compute_arviz_miness_runtime(chains, times)
            AP_TABLE[ϵ_ix, B_ix]  = avg_ap
    return ESS_TABLE, AP_TABLE

def cc_experiment_crwm(settings, tol=1e-11, rev_tol=1e-8, verbose=False):
    """Same as above but for C-RWM."""
    verboseprint = print if verbose else lambda *a, **k: None
    Bs = settings['Bs']
    u0s = settings['u0s']
    manifold = settings['manifold']
    N = settings['N']
    δ = settings['δ']
    n_chains = settings['n_chains']
    ESS_TABLE = zeros(len(Bs))
    AP_TABLE  = zeros(len(Bs))
    for B_ix, B in enumerate(Bs):
        chains = []
        times  = []
        avg_ap = 0.0
        for chain_ix in range(n_chains):
            start_time = time.time()
            s, e, a = zappa_sampling_storecomps_rattle_manifold(u0s[chain_ix, :], manifold, N, δ*B, B, tol=tol, rev_tol=rev_tol)
            runtime = time.time() - start_time
            verboseprint("B={} time={} a={}".format(B, runtime, a.mean()))
            chains.append(s)
            times.append(runtime)
            avg_ap += (a.mean() / n_chains)
        ESS_TABLE[B_ix] = compute_arviz_miness_runtime(chains, times)
        AP_TABLE[B_ix]  = avg_ap
    return ESS_TABLE, AP_TABLE